제너레이터는 전통적인 언어와 파이썬을 구분 짓는 또 다른 특징적인 기능이다. 제너레이터를 사용해 이상적으로 문제를 해결하는 방법과 제너레이터(또는 이터러블)를 파이썬스럽게 구현할 수 있는지 알아본다.

이터레이터 패턴을 따르면 왜 언어에서 자동으로 반복을 지원하게 되는지도 알게 될 것이다. 이 개념에서 출발하여 제너레이터가 어떻게 코루틴이나 비동기 프로그래밍 같은 기능을 지원하기 위한 기본 기능이 되었는지 살펴볼 것이다.

* 프로그램의 성능을 향상시키는 제너레이터 만들기
* 이터레이터가 파이썬에 어떻게 (특히 이터레이터 패턴을 사용하여) 완전히 통합되었는지 확인
* 이터레이션 문제를 이상적으로 해결하는 방법
* 제너레이터가 어떻게 코루틴과 비동기 프로그래밍의 기반이 되는 역할을 하는지 확인 
* 코루틴을 지원하기 위한 yield from, await, async def와 같은 문법의 세부 기능 확인

## 제너레이터 만들기

제너레이터는 파이썬에서 고성능이면서도 메모리를 적게 사용하는 반복을 위한 방법으로(PEP-255) 아주 오래 전 2001년에 소개되었다.

제너레이터는 한 번에 하나씩 구성요소를 반환해주는 이터러블을 생성해주는 객체이다. 제너레이터를 사용하는 주요 목적은 메모리를 절약하는 것이다. 거대한 요소를 한꺼번에 메모리에 저장하는 대신 특정 요소를 어떻게 만드는지 아는 객체를 만들어서 필요할 때마다 하나씩만 가져오는 것이다.

이 기능은 하스켈과 같은 다른 함수형 프로그래밍 언어가 제공하는 것과 비슷한 방식으로 게으른 연산(lazy computation)을 통해 무거운 객체를 사용할 수 있도록 한다. 게으른 연산의 특성을 가졌기 때문에 무한 시퀀스를 사용할 수도 있다.

### 제너레이터 개요

먼저 예제를 살펴보자. 지금 하려는 것은 대규모의 구매 정보에서 최저 판매가, 최고 판매가, 평균 판매가를 구하는 것이다.

문제의 단순화를 위해 두 개의 필드만 있는 CSV 파일이 있다고 가정해보자.

모든 구매 정보를 받아 필요한 지표를 구해주는 객체를 만들어보자. 최솟값이나 최댓값 같은 지표는 min(), max() 같은 내장 함수를 사용하여 쉽게 구할 수 있다. 그러나 어떤 지표는 단번에 구할 수 없고 모든 구매 이력을 반복해야만 한다.

지표를 구하는 코드 자체는 간단하다. for 루프의 각 단계에서 각 지표를 업데이트하기만 하면 된다. 일단 다음처럼 간단하게 구현을 하고 제너레이터에 대한 학습을 한 뒤에 훨씬 간단하고 깔끔한 형태의 구현을 다시 해볼 것이다.

In [84]:
class PurchasesStats:
    def __init__(self, purchases):
        self.purchases = iter(purchases)
        self.min_price: float = None
        self.max_price: float = None
        self._total_purchases_price: float = 0.0
        self._total_purchases = 0
        self._initialize()
    
    def _initialize(self):
        try:
            first_value = next(self.purchases)
        except:
            raise ValueError("no values provided")
        
        self.min_price = self.max_price = first_value
        self._update_avg(first_value)
    
    def process(self):
        for purchase_value in self.purchases:
            self._update_min(purchase_value)
            self._update_max(purchase_value)
            self._update_avg(purchase_value)
        return self
    
    def _update_min(self, new_value: float):
        if new_value < self.min_price:
            self.min_price = new_value
    
    def _update_max(self, new_value: float):
        if new_value > self.max_price:
            self.max_price = new_value
    
    @property
    def avg_price(self):
        return self._total_purchases_price / self._total_purchases
    
    def _update_avg(self, new_value: float):
        self._total_purchases_price += new_value
        self._total_purchases += 1
        
    def __str__(self):
        return (
            f"{self.__class__.__name__}({self.min_price}), "
            f"{self.max_price}, {self.avg_price})"
        )

이 객체는 모든 구매 정보를 받아서 필요한 계산을 한다. 이제 이 모든 정보를 로드해서 어딘가에 담아서 반환해주는 함수를 만들어보자. 다음은 첫 번째 버전이다.

In [49]:
def _load_purchases(filename):
    purchases = []
    with open(filname) as f:
        for line in f:
            *_, price_raw = line.partition(",")
            purchases.append(float(price_raw))
    return purchases

이 코드는 정상적인 결과를 반환한다. 파일에서 모든 정보를 읽어서 리스트에 저장한다. 그러나 성능에 문제가 있다. 파일에 상당히 많은 데이터가 있다면 로드하는데 시간이 오래 걸리고 메인 메모리에 담지 못할 만큼 큰 데이터일 수도 있다.

그런데 앞서 작성한 코드를 살펴보면 한 번에 하나의 데이터만을 사용하고 있다는 것 알 수 있다. 그렇다면 굳이 파일의 모든 데이터를 한 번에 모두 읽어 와서 메모리에 보관해야 할 이유가 무엇일까? 뭔가 더 좋은 해결책이 있을 것이다.

해결책은 제너레이터를 만드는 것이다. 파일의 전체 내용을 리스트에 보관하는 대신에 필요한 값만 그때그때 가져오는 것이다. 다음과 같이 코드를 수정한다.

In [50]:
def load_purchases(filename):
    with open(filename) as f:
        for line in f:
            *_, price_raw = line.partition(",")
            yield float(price_raw)

이렇게 수정하면 메모리 사용량이 급격하게 떨어지는 것을 볼 수 있다. 결과를 담을 리스트가 필요 없어졌으며 return 문 또한 사라졌다.

이 경우 load_purchases 함수를 제너레이터 함수 또는 단순히 제너레이터라고 부른다.

파이썬에서 어떤 함수라도 yield 키워드를 사용하면 제너레이터 함수가 된다. yeild가 포함된 이 함수를 호출하면 제너레이터의 인스턴스를 만든다.

모든 제너레이터 객체는 이터러블이다. 이터러블에 대해서는 뒤에 자세히 알아본다.

이터러블은 for 루프와 함께 사용할 수 있다는 정도만 알아두자. 여기서 주목할 점은 이 함수의 사용 코드가 그대로라는 점이다. 코드를 수정한 뒤에도 for 루프를 사용하여 지표를 계산하는 코드는 그대로이다. 

이터러블을 사용하면 for 루프의 다형성을 보장하는 이와 같은 강력한 추상화가 가능하다. 이터러블 인터페이스를 따르면 투명하게 객체의 요소를 반복하는 것이 가능하다.

### 제너레이터 표현식

제너레이터를 사용하면 많은 메모리를 절약할 수 있다. 또한 제너레이터는 이터레이터이므로 리스트나 튜플, 세트처럼 많은 메모리를 필요로 하는 이터러블이나 컨테이너의 대안이 될 수 있다. 

컴프리헨션에 의해 정의될 수 있는 리스트나 세트, 사전처럼 제너레이터도 제너레이터 표현식으로 정의할 수 있다. 물론 제너레이터 표현식을 제너레이터 컴프리헨션으로 불러야 한다는 주장도 있다. 

같은 방법으로 리스트 컴프리헨션을 사용해볼 것이다. 대괄호를 괄호로 교체하면 표현식의 결과로부터 제너레이터가 생성된다. 제너레이터 표현식은 sum()이나 max()와 같이 이터러블 연산이 가능한 함수에 직접 전달할 수도 있다.

In [51]:
[x**2 for x in range(10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [52]:
(x**2 for x in range(10))

<generator object <genexpr> at 0x07592030>

In [53]:
sum(x**2 for x in range(10))

285

## 이상적인 반복

파이썬에서 반복을 할 때 유용하게 사용할 수 있는 관용적인 코드를 살펴볼 것이다. 이러한 코드 예제는(특히 제너레이터 표현식을 알게 된 뒤에는) 제너레이터를 활용한는 방법을 익힐 수 있게 해주고 제너레이터와 관련된 전형적인 문제를 해결하는 방법을 알려준다. 

### 관용적인 반복 코드

우리는 이미 내장 함수인 enumerate()에 익숙하다. 이 함수는 이터러블을 입력 받아서 인덱스 번호와 원본의 원소를 튜플 형태로 변환하여 enumerate 객체를 반환한다.

In [54]:
list(enumerate('abcdef'))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]

좀 더 저수준에서 이와 유사한 객체를 만들어볼 것이다. 이 객체는 단순히 시작 값을 입력하면 무한 시퀀스를 만드는 역할을 한다.

다음과 같이 간단하게 만들 수 있다. 객체의 next() 함수를 호출할 때마다 다음 시퀀스 값을 무제한 출력해준다.

In [55]:
class NumberSequence:
    def __init__(self, start=0):
        self.current = start
    
    def next(self):
        current = self.current
        self.current += 1
        return current

이 인터페이스에 기반을 두어 클라이언트를 작성하면 명시적으로 next() 함수를 호출해야 한다.

In [56]:
seq = NumberSequence()
seq.next()

0

In [57]:
seq.next()

1

In [58]:
seq2 = NumberSequence(10)
seq2.next()

10

그러나 이 코드로 enumerate() 함수를 사용하도록 재작성할 수는 없다. 왜냐하면 일반 파이썬의 for 루프를 사용하기 위한 인터페이스를 지원하지 않기 때문이다. 이는 또한 이터러블 형태의 파라미터로는 사용할 수 없다는 것을 뜻한다. 다음 코드를 살펴보자.

In [59]:
list(zip(NumberSequence(), "abcdef"))

TypeError: zip argument #1 must support iteration

문제는 Numbersequence가 반복을 지원하지 않는다는 것이다. 이 문제를 해결하려면 \__iter__() 매직 메서드를 구현하여 객체가 반복 가능하게 만들어야한다. 또한 next() 메서드를 수정하여 \__next__ 매직 메서드를 구현하면 객체는 이터레이터가 된다.

In [60]:
class SequenceofNumbers:
    def __init__(self, start=0):
        self.current = start
    
    def __next__(self):
        current = self.current
        self.current += 1
        return current
    
    def __iter__(self):
        return self

이렇게 하면 요소를 반복할 수 있을 뿐 아니라, .next() 메서드를 호출할 필요도 없다. 왜냐하면 \__next__() 메서드를 구현헀으므로 next() 내장 함수를 사용할 수 있기 때문이다.

In [61]:
list(zip(SequenceofNumbers(), "abcdef"))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]

In [62]:
seq = SequenceofNumbers(100)

In [63]:
next(seq)

100

In [64]:
next(seq)

101

### next() 함수

next() 내장 함수는 이터러블을 다음 요소로 이동시키고 기존의 값을 반환한다.

In [65]:
word = iter('hello')
next(word)

'h'

In [66]:
next(word)

'e'

In [67]:
next(word)
next(word)
next(word)

'o'

In [68]:
next(word)

StopIteration: 

이터레이터가 더 이상의 값을 가지고 있지 않다면 StopIteration 예외가 발생한다.

이 예외는 반복이 끝났음을 나타내며 사용할 수 있는 요소가 더 이상  없음을 나타낸다.

이 문제를 해결하고 싶다면 StopIteration 예외를 캐치하는 것 외에도 next() 함수의 두 번째 파라미터에 기본 값을 제공할 수도 있다. 이 값을 제공하면 StopIertation을 발생시키는 대신 기본 값을 반환한다.

In [69]:
next(word, "default value")

'default value'

### 제너레이터 사용하기

앞의 코드는 제너레이터를 사용하여 훨씬 간단하게 작성할 수 있다. 제너레이터를 사용하면 클래스를 만드는 대신 다음과 같이 필요한 값을 yield하는 함수를 만들면 된다.

In [70]:
def seqeunce(start=0):
    while True:
        yield start
        start += 1

함수의 본문에 있는 yield 키워드가 해당 함수를 제너레이터로 만들어 준다는 것에 유의하자. 이 함수는 제너레이터이기 때문에 위처럼 무한 루프를 사용해도 완벽하게 안전하다. 제너레이터 함수가 호출되면 yield 문장을 만나기 전까지 실행된다. 그리고 값을 생성하고 그 자리에서 멈춘다.

In [71]:
seq = seqeunce(10)

In [72]:
next(seq)

10

In [73]:
next(seq)

11

In [74]:
next(seq)

12

In [75]:
list(zip(seq, "abcdef"))

[(13, 'a'), (14, 'b'), (15, 'c'), (16, 'd'), (17, 'e'), (18, 'f')]

In [76]:
def seq(start=0):
    while True:
        yield start
        start += 1
        if start == 3:
            break

In [77]:
seq = seq()

In [78]:
next(seq)

0

In [79]:
next(seq)

1

In [80]:
next(seq)

2

In [81]:
next(seq, 10)

10

### Itertools 

이터러블로 작업하면 코드가 파이썬 자체와 더 잘 어울리는 장점이 있다. 왜냐하면 이터레이션이 언어의 중요한 컴포넌트이기 때문이다. 또란 itertools 모듈(ITER-01)을 사용하면 그 기능을 온전히 활용할 수 있다. 사실 방금 사용한 sequence() 제너레이터는 itertools.count()와 상당히 유사하다. 그러나 itertools 모듈에는 여러 추가 기능이 있다.

이터레이터, 제너레이터, itertools와 관련하여 가장 멋진 것 중에 하나는 이들을 서로 연결하여 새로운 객체를 만들 수 있다는 것이다.

예를 들어 처음에 예제로 돌아가서 구매 이력에서 지표를 계산하는 과정을 다시 살펴보자. 만약 특정 기준을 넘은 값에 대해서만 연산을 하려면 어떻게 해야 할까? 가장 간단한 방법은 while 문 안에 조건을 추가하는 것이다.

In [82]:
def process(self):
    for purchase in self.purchases:
        if purchase > 1000.0:
            ...

그러나 이 방법은 파이썬스럽지 않을 뿐 아니라 너무 엄격하다. 너무 엄격하다는 것은 나쁜 코드를 의미한다. 이것은 수정 사항을 잘 반영할 수 없다. 만약 기준 수치가 변경된다면 파라미터로 전달해야 할까? 만약 파라미터가 둘 이상 필요하다면 어떻게 할까? 만약 조건이 특정 기준 이하가 되면 어떻게 할까? 람다 함수를 사용해야 할까?

사실 이번 객체가 이 질문에 대한 답변을 할 필요는 없다. 이 객체의 고유 책임은 구매 이력에 대해 잘 정의된 지표 값을 계산하고 출력하는 것뿐이다. 이러한 요구사항을 이번 객체에 반여하는 것은 큰 실수이다. 다시 한 번 말하지만 클린 코드는 융통성이 잇어야 하고 외부 요인에 결합력이 높아서는 안 된다. 이러한 요구사항은 다른 곳에서 해결되어야 한다.

이 객체는 클라이언트의 요구로부터 독립되어야 한다. 이 클래스의 책임이 작을수록 클라이언트는 재사용성이 높아지믈 보다 유용하게 된다.

코드를 수정하는 대신 그대로 유지하고 클라이언트 클래스 요구사항이 무엇이든 그에 맞게 필터링하여 새로운 데이터를 만든다고 가정하자.

예를 들어 1,000개 넘게 구매한 이겨의 처음 10개만 처리하려고 하면 다음과 같이 하면 된다.

In [93]:
from itertools import islice
purchases = [995+i for i in range(20)] # 테스트를 위해 생성함 이 객체는 메모리에 올라감 
purchases = islice(filter(lambda p:p > 1000.0, purchases), 10)
stats = PurchasesStats(purchases).process()
print(stats)

PurchasesStats(1001), 1010, 1005.5)


이런 식으로 필터링을 해도 메모리의 손해는 없다. 왜냐하면 모든 것이 제너레이터이므로 게으르게 평가된다. 즉 마치 전체에서 필터링한 값으로 연산을 한 것처럼 보이지만, 실제로는 하나씩 가져와서 모든 것을 메모리에 올릴 필요가 없는 것이다.

### 이터레이터를 사용한 코드 간소화

지금까지 이터레이터 또는 itertools 모듈을 활용해 코드를 개선하는 것을 살펴보았다. 각 사례의 최적화 방법에 대한 결론도 추론할 수 있다.

### 여러 번 반복하기

이제 이터레이터에 대해 좀 더 깊이 있게 살펴보았고 itertools 모듈까지 확인하였으므로 이 장의 처음에 살펴보았던 예제를 훨씬 간소화할 수 있다.

In [106]:
import itertools

def process_purchases(purchases):
    min_, max_, avg = itertools.tee(purchases, 3)
    print(min_, max_, avg)
    return min(min_), max(max_), median(avg)

In [107]:
purchases = [995+i for i in range(20)] # 테스트를 위해 생성함 이 객체는 메모리에 올라감 
print(process_purchases(purchases))

<itertools._tee object at 0x071A05F8> <itertools._tee object at 0x07493328> <itertools._tee object at 0x07493AF8>
(995, 1014, 1004.5)


itertools.tee*는 원래의 이터러블을 세 개의 새로운 이터러블로 분할한다. 그리고 구매 이력을 세 번 반복할 필요없이 분할된 이터러블을 사용해 필요한 연산을 할 것이다. 

### 중첩 루프

경우에 따라 1차원 이상을 반복해서 값을 찾아야 할 수 있다. 가장 쉽게 해결하는 방법으로 중첩 루프가 떠오른다. 값을 찾으면 순환을 멈추고 break 키워드를 호출해야 하는데 이런 경우 한 단계가 아니라 두 단계 이상을 벗언나야 하므로 정상적으로 동작하지 않는다.

플래그나 예외를 발생시켜야 할까? 아니다. 플래그와 마찬가지이고 오히려 더 나쁘다. 왜냐하면 예외는 로직을 제어하기 위한 수단이 아니기 때문이다. 코드를 잘게 나누어 함수에서 반환해야 할까? 비슷했지만 완벽한 방법은 아니다.

가장 좋은 방법은 가능하면 중첩을 풀어서 1차원 루프로 만드는 것이다.

다음은 피해야 할 코드이다.

In [108]:
def search_nested_bad(array, desired_value):
    coords = None
    for i, row in enumrate(array):
        for j, cell in enumerate(row):
            if cell == desired_value:
                coords = (i, j)
                break
        if coords is not None:
            break
    
    if coords is None:
        raise ValueError(f"{desired_value} not found")
    
    logger.info("[%i, %i]에서 값 %r 찾음", *coords, desired_value)
    return coords

다음은 종료 플래그를 사용하지 않은 보다 간단하고 컴팩트한 형태의 예이다.

In [109]:
def _iterate_array2d(array2d):
    for i, row in enumerate(array2d):
        for j, cell in enumerate(row):
            yield (i, j), cell

def search_nested(array, desired_value):
    try:
        coord = next(coord for (coord, cell) in _iterate_array2d(array) if cell == desired_value)
    except StopIteration:
        raise ValueError("{desired_value} not found")
        
    logger.info("[%i, %i]에서 값 %r 찾음", *coords, desired_value)
    return coord

보조 제너레이터가 어떻게 반복을 추상화했는지 살펴볼 수 있다. 지금은 2차원 배열을 사용했으나 나중에 더 많은 차원의 배열을 사용하는 경우에도 클라이언트는 그서에 대해 알 필요가 없이 기존 코드를 그대로 사용하면 된다. 이것이 다음 섹션에서 설명할 이터레이터 디자인 패턴의 본질이다. 파이썬은 이터레이터 객체를 지원하므로 자동으로 투명해진다.

* 최대한 중첩 루프를 제거하고 추상화하여 반복을 단순화한다.

### 파이썬의 이터레이터 패턴

파이썬에서의 반복을 좀 더 자세히 이해하기 위해 제너레이터로부터 약간 벗어나볼 것이다. 제너레이터는 이터러블 객체의 특별한 경우이지만 파이썬의 반복은 제너레이터 이상의 것으로 훌륭한 이터러블 객체를 만들게 되면 보다 효율적이고 컴팩트하고 가독성이 높은 코드를 작성할 수 있게 된다.

앞의 코드에서는 이터러블 객체이면서 또한 이터레이터인 객체를 살펴보았다. 이터레이터는 \__iter__()와 \__next__() 매직 메서드를 구현한 객체이다. 일반적으로 이렇게 구현을 하지만 엄밀히 말하면 항상 이 두 가지를 꼭 구현해야 하는 것은 아니다. 여기서는 \__iter__를 구현한 이터러블 객체와 \__next__를 구현한 이터레이터 객체를 비교해볼 것이다.

또한 시퀀스와 컨테이너 객체의 반복에 대해서도 다룰 것이다.

### 이터레이션 인터페이스

이터러블은 반복을 지원하는 객체로 크게 보면 아무 문제없이 for ... in ... 루프를 실행할 수 있다는 것을 뜻한다. 그러나 이터러블과 이터레이터는 다르다.

일반적으로 이터러블은 반복할 수 있는 어떤 것으로 실제 반복을 할 때는 이터레이터를 사용한다. 즉 \__iter__ 매직 메서드를 통해 이터레이터를 반환하고, \__next__ 매직 메서드를 통해 반복 로직을 구현하는 것이다.

이터레이터는 단지 내장 next() 함수 호출 시 일련의 값에 대해 한 번에 하나씩만 어떻게 생성하는지 알고 있는 객체이다. 이터레이터를 호출하지 않은 상태에서 다음 값을 요청 받기 전까지 그저 얼어있는 상태일 뿐이다. 이러한 의미에서 모든 제너레이터는 이터레이터이다. 

|파이썬 개념|매직 메서드|비고|
|------------------|--------------------|------|
|이터러블| \__iter__ | 이터레이터와 함께 반복 로직을 만든다. 이것을 구현한 객체는 for ... in ... 구문에서 사용할 수 있다.|
|이터레이터| \__next__ | 한 번에 하나씩 값을 생산하는 로직을 정의한다. 더 이상 생산할 값이 없을 경우 StopIteration 예외를 발생시킨다. 내장 next() 함수를 사용해 사나씩 값을 읽어 올 수 있다.|

다음 코드는 이터러블하지 않은 이터레이터 객체의 예이다. 이것은 오직 한 번에 하나씩 값을 가져올 수만 있다. 여기서 sequence는 잠시 후 살펴볼 파이썬의 시퀀스가 아니고 일련의 연속된 숫자를 나타낸다.

In [114]:
class SequenceIterator:
    def __init__(self, start=0, step=1):
        self.current = start
        self.step = step
    
    def __next__(self):
        value = self.current
        self.current += self.step
        return value

시퀀스에서 하나씩 값을 가져올 수 있지만 반복할 수는 없다(무한 루프를 유발하므로 어쩌면 다행이다).

In [115]:
si = SequenceIterator(1, 2)
next(si)

1

In [116]:
next(si)

3

In [117]:
next(si)

5

In [118]:
for _ in SequenceIterator(): pass

TypeError: 'SequenceIterator' object is not iterable

객체가 \__iter__() 메서드를 구현하지 않았으므로 에러가 발생하는 것이 당연하다.

단지 설명을 하자면 반복을 다른 객체로 분리할 수도 있다.

### 이터러블이 가능한 시퀀스 객체

앞에서 보았듯이 \__iter__() 매직 메서드를 구현한 객체는 for 루프에서 사용할 수 있다. 이것은 큰 특징이지만 꼭 이런 형태여야만 반복이 가능한 것은 아니다. 파이썬이 for 루프를 만나면 객체가 \__iter__를 구현했는지 확인하고 있으면 그것을 사용한다. 그러나 없을 경우는 다른 대비 옵션을 가동한다.

객체가 시퀀스인 경우(즉 \__getitem__()과 \__len__() 매직 메서드를 구현한 경우)도 반복 가능하다. 이 경우 인터프리터는 IndexError 예외가 발생할 때까지 순서대로 값을 제공한다. 

IdexError는 StopIteration과 유사하게 반복에 대한 중지를 알리는 역할을 한다.

이러한 동작을 확인하기 위해 특정 숫자 범위에 대해 map()을 구현한 시퀀스 객체를 살펴보자.

In [125]:
class MappedRange:
    """특정 숫자 범위에 대해 맵으로 변환"""
    
    def __init__(self, transformation, start, end):
        self._transformation = transformation
        self._wrapped = range(start, end)
        
    def __getitem__(self, index):
        value = self._wrapped.__getitem__(index)
        result = self._transformation(value)
        print("Index %d: %s" %(index, result))
        return result
    
    def __len__(self):
        return len(self._wrapped)

이 예제는 오직 일반 for 루프를 사용해 반복 가능하다는 것을 보여주기 위한 것이다. \__getitem__ 메서드에서는 객체가 반복하는 동안 어떤 값이 전달되었는지 확인하기 위해 로그를 출력한다.

In [126]:
mr = MappedRange(abs, -10, 5)

In [127]:
mr[0]

Index 0: 10


10

In [128]:
mr[-1]

Index -1: 4


4

In [129]:
for i in mr:
    print(i)

Index 0: 10
10
Index 1: 9
9
Index 2: 8
8
Index 3: 7
7
Index 4: 6
6
Index 5: 5
5
Index 6: 4
4
Index 7: 3
3
Index 8: 2
2
Index 9: 1
1
Index 10: 0
0
Index 11: 1
1
Index 12: 2
2
Index 13: 3
3
Index 14: 4
4


In [130]:
list(mr)

Index 0: 10
Index 1: 9
Index 2: 8
Index 3: 7
Index 4: 6
Index 5: 5
Index 6: 4
Index 7: 3
Index 8: 2
Index 9: 1
Index 10: 0
Index 11: 1
Index 12: 2
Index 13: 3
Index 14: 4


[10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4]

In [131]:
mr

다만 이러한 방법이 있다는 것을 알아두는 것은 좋지만 객체가 \__iter__를 구현하지 않았을 때 동작하는 대비책임에 주의하자. 따라서 대부분의 경우 단순히 반복 가능한 객체를 만드는 것이 아니라 적절한 시퀀스를 만들어 해결하는 것이 바람직하다. 

## 코루틴(coroutine)

이미 알고 있는 것처럼 제너레이터는 반복 가능한 객체로 \__iter__()와 \__next__()를 구현한다. 이를 사용한 프로토콜은 파이썬에 의해 자동으로 제공되므로 제너레이터 객체를 사용하면 next() 함수를 통해 다음 요소로 반복 또는 이동이 가능하다.

이러한 기본 기능 외에도 제너레이터를 코루틴(PEP-342)으로 활용할 수 있다. 여기서는 비동기 프로그래밍의 기초를 지원하기 위해 제너레이터가 어떻게 코루틴으로 진화하는지 살펴보고, 다음 섹션에서 비동기 기능을 위한 파이썬의 새로운 기능과 문법에 대해서 살펴보겠다. 코루틴을 지원하기 위해 PEP-342에 추가된 기본 메서드는 다음과 같다.

* .close()
* .throw(ex_type[, ex_value[, ex_traceback]])
* .send(value)

### 제너레이터 인터페이스의 메서드

이 섹션에서는 앞서 언급한 각각의 메서드가 무엇인지, 어떻게 작동하는지 그리고 어떻게 사용되는지 알아볼 것이다. 메서드의 사용방법을 이해한 뒤에 간단한 코루틴을 만들어볼 것이다.

그리고 코루틴의 고급 사용법과 서브 제너레이터(코루틴)에 위임을 통해 리팩토링하는 방법, 여러 코루틴을 조합하는 방법에 대해서도 알아볼 것이다.

### close()

이 메서드를 호출하면 제너레이터에서 GeneratorExit 예외가 발생한다. 이 예외를 따로 처리하지 않으면 제너레이터가 더 이상 값을 생성하지 않으며 반복이 중지된다.

이 예외는 종료 상태를 지정하는데 사용될 수 있다. 코루틴이 일종의 자원 관리를 하는 경우 이 예외를 통해서 코루틴이 보유한 모든 자원을 ㅎ제할 수 있다. 일반적으로 컨텍스트 관리자를 사용하거나 finally 블록에 코드를 배치하는 것과 비슷하지만 이 예외를 사용하면 보다 명확하게 처리할 수 있다.

다음 예제는 코루틴을 사용하여 데이터베이스 연결을 유지한 상태에서 한 번에 모든 레코드를 읽는 대신에 특정 크기의 페이지를 스트리밍한다.

In [132]:
def stream_db_records(db_handler):
    try:
        while True:
            yield db_handler.read_n_records(10)
    except GeneratorExit:
        db_handler.close()

제너레이터를 호출할 때마다 데이터에비스 핸들러에서 얻은 10 개의 레코드를 반환하고, 명시적으로 반복을 끝내고 close()를 호출하면 데이터베이스 연결도 함께 종료한다.

In [134]:
streamer = stream_db_records(DBHandler('testdb'))
next(streamer)
next(streamer)
streamer.close()

제너레이터에서 작업을 종료할 떄는 close() 메서드를 사용한다.

### throw(ex_type[, ex_value[, ex_traceback]])

이 메서드는 현재 제너레이터가 중단된 위치에서 예외를 던진다. 제너레이터가 예외를 처리했으면 해당 except 절에 있는 코드가 호출되고, 예외를 처리하지 않았으면 예외가 호출자에게 전파된다. 

여기서는 코루틴이 예외를 처리했을 때와 그렇지 않을 때의 차이를 설명하기 위해 코드를 약간 수정했다.

In [57]:
class CustomException(Exception):
    pass
    
    def stream_data(db_handler):
        while True:
            try:
                yield db_handler.read_n_records(10)
            except CustomException as e:
                logger.warning("처리 가능한 에러 %r, 계속 진행", e)
                print('처리 가능한 에러 %r, 계속 진행' %(e))
                
            except Exception as e:
                db_handler.close()
                break

이제 CusmtomException을 처리하고 있으며 이 예외가 발생한 경우 제너레이터는 INFO 레벨의 메세지를 기록한다. 물론 비즈니스 로직에 따라 다르게 구현할 수 있다. 그리고 다음 yield 구문으로 이동하여 데이터베이스에서 다시 데이터를 가져온다.

이 예제는 모든 예외를 처리하고 있지만 마지막 블록이 없으면 제너레이터가 중지된 행에서 예외가 호출자에게 전파되고 제너레이터는 중지된다.

### send(value)

앞의 예제에서는 데이터베이스 레코드를 조회하는 간단한 제너레이터를 만들고 반복을 끝낼 때 데이터베이스 리소스를 해제했다. 이것은 제너레이터가 제공하는 close 메서드를 사용하는 좋은 예제이지만 코루틴으로 보다 많은 일을 할 수 있다.

현재 제너레이터의 주요 기능은 고정된 수의 레코드를 읽는 것이다. 이제 읽어올 개수를 파라미터로 받아서 호출하도록 수정해보자. 안타깝게도 next() 함수는 이러한 옵션을 제공하지 않는다. 이럴 떄 send() 메서드를 사용하면 된다. 

In [137]:
def stream_db_records(db_handler):
    retrieved_data = None
    previous_page_size = 10
    try:
        while True:
            page_size = yield retrieved_data
            if page_size is None:
                page_size = previous_page_size
            previous_page_size = page_size
            
            retrieved_data = db_handler.read_n_records(page_size)
    except GeneratorExit:
        db_handler.close()

이제 send() 메서드를 통해 인자 값을 전달할 수 있다. 사실 이 메서드는 제너레이터와 코루틴을 구분하는 기준이 된다. send() 메서드를 사용했다는 것은 yield 키워드가 할당 구문의 오른쪽에 나오게 되고 인자 값을 받아서 다른 곳에 할당할 수 있음을 뜻한다.

코루틴에서는 일반적으로 다음과 같은 형태로 yield 키워드를 사용한다.

In [139]:
receice = yield produced

호출자는 next() 메서드를 호출하여 다음 라운드가 되었을 때 값을 가져올 수 있다. 다른 하나는 거꾸로 호출자로부터 send() 메서드를 통해 전달된 produced 값을 받는 것이다. 이렇게 입력된 값은 receive 변수에 할당된다.

코루틴에 값을 전송하는 것은 yield 구문이 멈춘 상태에서만 가능하다. 그렇게 되려면 일단 코루틴을 해당 상태까지 이동시켜야 한다. 코루틴이 해당 상태로 이동하는 유일한 방법은 next()를 호출하는 것이다. 즉 코루틴에게 무엇인가를 보내기 전에 next() 메서드를 적어도 한 번은 호출해야 한다는 것을 의미한다. 그렇지 않으면 TypeError가 발생한다.

* 코루틴에서 send() 메서드를 호출하려면 항상 next()를 먼저 호출해야 한다.

* 코루틴은 cooperative routine의 약자로 일반적으로 알고 있는 함수나 메서드 같은 서브루틴이 메인루틴과 종속관계를 가진 것과 다르게, 메인루틴과 대등한 관계로 협력하는 모습에서 코루틴이라고 불리게 되었다.

다시 예제로 돌아가서 데이터베이스에서 읽을 레코드의 길이를 파라미터를 받도록 수정해보자. 제너레이터에서 처음 next()를 호출하면 yield를 포함하는 위치까지 이동한다. 그리고 현재 샅애의 변수 값을 반환하고 거기에 멈춘다. 변수의 초기 값이 None이므로 처음 next()를 호출할면 None을 반환한다. 여기에서 두 가지의 옵션이 있다. 그냥 next()를 호출하면 기본값인 10을 사용하여 평소처럼 이후 작업이 계속된다. next()는 send(None)과 같기 때문에 if page_size is None에서 기본 값을 사용하도록 설정된다.

반면에 send(value)를 통해 명시ㅣ적인 값을 제공하면 yield 문의 반환 값으로 page_size 변수에 설정된다. 이제 기본 값이 아닌 사용자가 지정한 값이 page_size로 설정되고 해당 크기만큼 데이터베이스에서 레코드를 읽어오게 된다.

이어지는 호출에 대해서도 같은 로직이 적용된다. 중요한 것이 이제 아무 때나 페이지 크기를 지정할 수 있다는 점이다.

이제 동작 원리를 이해했으므로 많은 파이썬 개발자는 보다 간편한 버전을 기대할 것이다. 파이썬은 항상 간결하고 작은 깔금한 코드를 자향한다.

In [2]:
def stream_db_records(db_handler):
    retrieved_data = None
    page_size = 10
    try:
        while True:
            page_size = (yield retrieved_data) or page_size
            retrieved_data = db_handler.read_n_records(page_size)
    except GeneratorExit:
        db_handler.close()

이 코드는 보다 간결할 뿐 아니라 이해하기도 쉽다. yield 주변의 괄호는 해당 문장이 함수를 호출하는 것 처럼 사용되고 page_size와 비교할 것이라는 점을 명확히 한다.

이것은 기대한 것처럼 동작하지만 send() 전에 next()를 먼저 호출해햐 한다는 것을 꼭 기억해야 한다. 그렇지 않으면 TypeError가 발생한다. 이러한 사전 작업은 지금 하려는 작업과는 상관이 없는 준비 작업일 뿐이다.

next()를 반드시 호출해야 한다는 것을 기억할 필요 없이 코루틴을 생성하자마자 바로 사용할 수 있다면 훨씬 편할 것이다. PYCOOK의 저자는 이를 해결해줄 흥미로운 데코레이터를 고안했다. 이 데코레이터의 목적은 코루틴을 좀 더 편리하게 하는 것으로 다음과 같이 자동으로 초기화를 해준다.

In [4]:
@prepare_coroutine
def stream_db_records(db_handler):
    retrieved_data = None
    page_size = 10
    try:
        while True:
            page_size = (yield retrieved_data) or page_size
            retrieved_data = db_handler.read_n_records(page_size)
    except GeneratorExit:
        db_handler.close()

앞의 예제에 prepare_coroutine()을 사용하여 next()를 호출하지 않고도 코루틴을 바로 사용할 수 있게 되었다.

yield retrieved_data 자체가 인자 값을 받는 것을 의미한다. 꼭 할당문이 있어야 변수를 받는 것은 아니다. 따라서 page_size = (yield retrieved_data) or page_size 문장은 input = (yield retrieved_data); page_size = input or page_size; 처럼 두 개의 문장으로 작성한 것과 같다.

### 코루틴 고급 주제

지금까지 코루틴에 대해 자세히 알아보았으며 간단한 예제도 만들어보았다. 이러한 코루틴은 사실 진보된 제너레이터라고 할 수 있다. 코루틴은 멋진 제너레이터이다. 그러나 실제로 보다 복잡한 코루틴을 병렬로 실행하려면 추가 기능이 필요하다. 

많은 코루틴을 처리하다보면 새로운 문제가 발생한다. 애플리케이션의 로직이 복잡해지면 예외 처리는 물론이고 서브 코루틴의 값을 어디에서든 사용하도록 해야 하고 여러 코루틴을 스케줄링 해야 한다. 이러한 일을 더 간편하게 하기 위해 제너레이터를 더 확장해야 했다. PEP-380에서 yield from이라는 새로운 생성자 구문을 도입하여 문제를 해결했다.

### 코루틴에서 갑 반환하기

이 장의 앞에서 설명한 것처럼 반복이란 StopIteration 예외가 발생할 때까지 next() 메서드를 계속해서 호출하는 메커니즘을 말한다. 

지금까지는 한 번에 하나씩 값을 생성하는 제너레이터의 특성을 확인했으며 일반적으로 for 루프의 모든 단계에서 생성되는 각 값에 대해서만 신경을 쓴다. 제너레이터에서 이것은 매우 논리적인 사고방식이지만 코루틴에서는 조금 다르다. 코루틴은 기술적으로는 제너레이터이지만 반복을 염두에 두고 만든 것이 아니라 나중에 코드가 실행될 때까지 코드의 실행을 멈추는 것을 목표로 한다.

코루틴의 디자인에서 이것은 흥미로운 부분이다. 코루틴은 일반적으로 반복보다는 상태를 중단하는데 초점을 맞추고 있다. 오히려 반복을 목적으로 코루틴을 만드는 것이 이상한 경우일 것이다. 파이썬에서 코루틴은 기술적으로 제너레이터에 기반을 두고 있기 때문에 이 둘의 개념이 쉽게 섞일 수 있다.

코루틴을 사용하여 정보를 처리하고 실행을 일시 중단하는 경우 경량 스레드(또는 다른 플랫폼에서는 그린 스레드)라고 생각하는 것이 좋다. 그렇다면 다른 정규 함수를 호출하는 것처럼 값을 반환하는 것이 이해가 된다. 

그러나 제너레이터가 일반 함수는 아니므로 value = generator()라고 하는 것은 제너레이터 객체를 만드는 것 외에는 아무것도 하지 않는다. 제너레이터가 값을 반환하게 하려면 어떻게 해야 할까? 반복을 하면 값을 가져올 수 있다.

제너레이터에서 값을 반환(return)하면 반복이 즉시 중단된다. 즉 더 이상 반복을 할 수 없다. 본래의 의미 체계를 유지하기 위해 StopIteration 예외가 발생해도 예외 객체 내에 반환 값이 저장되어 있다. 예외에서 해당 값을 처리하는 것은 호출자의 책임이다.

다음 예제는 제너레이터를 사용해 두 개의 값을 생성하고 세 번째 값을 반환한다. 마지막 return되는 값을 구하기 위해 예외를 처리하는 방법과 예외에서 어떻게 값을 구하는지 유의해서 살펴보자.

In [5]:
def generator():
    yield 1
    yield 2
    return 3

In [6]:
value = generator()

In [7]:
next(value)

1

In [8]:
next(value)

2

In [9]:
try:
    next(value)
except StopIteration as e:
    print(">>>>>> returned value ", e.value)

>>>>>> returned value  3


### 작은 코루틴에 위임하기 - yield from 구문

코루틴(제너레이터)이 값을 반환할 수 있다는 점은 다른 활용 가능성을 열어준 측면에서 흥미로운 기능이다. 그러나 값을 반환하는 기능 자체는 언어에서 지원해주지 않으면 조금 귀찮은 부분이 있다.

이 부분을 개선해주기 위한 구문이 바로 yield from이다. 이후 살펴볼 여러 기능과 함께 다른 하위 제너레이터에서 반환된 값을 수집하는 기능이 있다. 이전에 제너레이터가 값을 반환하는 것은 멋진 기능이지만 불행히도 value = generator()와 같은 문장은 동작하지 않았던 것을 기억하는가? 이제 value = yield from generator()와 같이 작성하면 그것이 가능하다.

### 가장 간단한 yield from 사용 예

가장 간단한 형태의 yield from 구문은 제너레이터 체인에서 살펴볼 수 있다. 제너레이터 체인은 여러 제너레이터를 하나의 제너레이터로 합치는 기능을 하는데 중첩된 for 루프를 사용해 하나씩 모으는 대신에 서브 제너레이터의 값을 한 번에 수집할 수 있게 해준다.

대표적인 예로 표준 라이브러리인 itertools.chain()과 비슷한 함수를 만들어보자. 이 함수는 여러 개의 이터러블을 받아서 하나의 스트림으로 반환한다.

다음과 같이 간단히 구현해볼 수 있다.

In [13]:
def chain(*iterables):
    for it in iterables:
        for value in it:
            yield value

여러 이터러블을 받아서 모두 이동한다. 모두 이터러블이므로 for ... in 구문을 지원하므로 개별 값을 구하려면 중첩 루프를 사용하면 된다. 이렇게 하면 리스트를 튜플과 비교하는 것처럼 직접 비교가 어려운 자료형에 대해서도 한 번에 처리할 수 있으므로 편리하다.

여기서는 yield from 구문을 사용하면 서브 제너레이터에서 직접 값을 생산할 수 있으므로 중첩 루프를 피할 수 있다. yield from 구문을 사용해 다음과 같이 코드를 단순화할 수 있다.

In [104]:
def yield_from_chain(*iterables):
#     print(iterables)
    for it in iterables:
        yield from it

In [105]:
yfc = yield_from_chain('123', '456', '789')

In [21]:
list(chain('hello', ['world'], ('tuple', 'of', 'values.')))

['h', 'e', 'l', 'l', 'o', 'world', 'tuple', 'of', 'values.']

In [22]:
list(yield_from_chain('hello', ['world'], ('tuple', 'of', 'values.')))

('hello', ['world'], ('tuple', 'of', 'values.'))


['h', 'e', 'l', 'l', 'o', 'world', 'tuple', 'of', 'values.']

In [23]:
list(yield_chain('hello', ['world'], ('tuple', 'of', 'values.')))

('hello', ['world'], ('tuple', 'of', 'values.'))


['hello', ['world'], ('tuple', 'of', 'values.')]

yield from 구문은 어떤 이터러블에 대해서도 동작하며 이것을 사용하면 마치 최상위 제너레이터가 직접 값을 yield한 것과 같은 효과를 나타낸다. 

yield from은 어떤 형태의 이터러블에서도 동작하므로 제너레이터 표현식도 마찬가지이다. 이제 yield from 구문을 활용해 입력된 파라미터의 모든 제곱지수를 만드는 제너레이터를 만들어보자. 예를 들어 all_powers(2, 3)은 2^0, 2^1. 2^2. 2^3을 생산한다.

In [24]:
def all_powers(n, pow):
    yield from (n ** i for i in range(pow+1))

이렇게 하면 기존의 서브 제너레이터에서 for 문을 사용해 값을 생산하는 대신 한 줄로 직접 값을 생산할 수 있으므로 편리하지만, 이것만으로 yield from을 언어에서 지원해야만 하는 이유였다고 보기는 어렵다.

사실 위와 같은 동작은 의도하지 않은 부가적인 효과이고 yield from의 진짜 존재의 이유는 다음 두 섹션에서 설명한다.

### 서브 제너레이터에서 반환된 값 구하기

다음 예제는 수열을 생산하는 두개의 중첩된 제너레이터를 호출한다. 각각의 제너레이터는 값을 반환하는데 최상위 제너레이터는 쉽게 반환 값을 확인할 수 있다. 바로 yield from 구문을 사용했기 때문이다. 

In [31]:
import logging
logger = logging.getLogger('sequence')

def sequence(name, start, end):
    logger.info('%s 제너레이터 %i에서 시작', name, start)
    print('%s 제너레이터 %i에서 시작' %(name, start))
    yield from range(start, end)
    logger.info("%s 제너레이터 %i에서 종료", name, end)
    print("%s 제너레이터 %i에서 종료" %(name, end))

    return end

def main():
    step1 = yield from sequence("first", 0, 5)
    step2 = yield from sequence("second", step1, 10)
    return step1 + step2

In [32]:
g = main()

In [33]:
next(g)

first 제너레이터 0에서 시작


0

In [34]:
next(g)

1

In [35]:
next(g)

2

In [36]:
next(g)

3

In [37]:
next(g)

4

In [38]:
next(g)

first 제너레이터 5에서 종료
second 제너레이터 5에서 시작


5

In [39]:
next(g)

6

In [40]:
next(g)

7

In [41]:
next(g)

8

In [42]:
next(g)

9

In [43]:
next(g)

second 제너레이터 10에서 종료


StopIteration: 15

main 제너레이터의 첫 번째 행은 내부 제너레이터로 위임하여 생산된 값을 가져온다. 이것은 이미 살펴본 것으로 새로운 내용은 아니다. sequence() 제너레이터 종료 시 반환 값을 step1으로 받아와서 다음 sequece() 제너레이터에 전달하는 부분을 유심히 살펴보자.

두 번째 제너레이터 역시 종료 시 값(10)을 반환하고 그러면 main 제너레이터는 이 두 결과의 합 (5+10=15)을 반환한다. 이 값은 StopIteration에 포함된 값이다. 

yield from을 사용하면 코루틴 종료 시 최종 반환 값을 구할 수 있다.

### 서브 제너레이터와 데이터 송수신하기

이제 코루틴의 진정한 강력함을 느낄 수 있게 해주는 멋진 기능을 살펴보자. 이미 소개한 것처럼 제너레이터는 코루틴처럼 동작할 수 있다. 값을 전송하고 예외를 던지면 코루틴 역할을 하는 해당 제너레이터는 값을 받아서 처리하거나 반드시 예외를 처리해야 한다. 

앞의 예제처럼 서브 제너레이터에 위임한 코루틴에 대해서도 마찬가지이다. 그런데 수동으로 이런 것들을 처리하면 매우 복잡할 것이다. yield from에서 자동으로 처리하지 않을 경우 직접 처리하는 코드는 PEP-380에서 확인할 수 있다.

앞의 예제에서 최상위 main 제너레이터는 그대로 유지하고, 값을 수신하고 예외를 처리할 내부 제너레이터인 sequence 함수를 수정한다. 다음 코드는 이상적인 코드는 아니고 동작 메커니즘을 설명하기 위한 용도의 코드이다.

In [44]:
def sequence(name, start, end):
    value = start
    logger.info("%s 제너레이터 %i에서 시작", name, value)
    print("%s 제너레이터 %i에서 시작" %(name, value))
    while value < end:
        try:
            received = yield value
            logger.info("%s 제너레이터 %r 값 수신", name, received)
            print("%s 제너레이터 %r 값 수신" %(name, received))
            value += 1
        except CustomException as e:
            logger.info("%s 제너레이터 %s 에러 처리", name, e)
            print("%s 제너레이터 %s 에러 처리" %(name, e))
            received = yield "OK"
    return end

def main():
    step1 = yield from sequence("first", 0, 5)
    step2 = yield from sequence("second", step1, 10)
    return step1 + step2

이제 main 코루틴을 반복하는 것 뿐 아니라 내부 sequence 제너레이터에서 어떻게 처리하는지 확인하기 위해 값을 전달하거나 예외를 던져본다.

In [94]:
g = main()

In [95]:
next(g)

first 제너레이터 0에서 시작


0

In [96]:
next(g)

first 제너레이터 None 값 수신


1

In [97]:
g.send("첫 번째 제너레이터를 위한 인자 값")

first 제너레이터 '첫 번째 제너레이터를 위한 인자 값' 값 수신


2

In [98]:
g.throw(CustomException("처리 가능한 예외 던지기"))

first 제너레이터 처리 가능한 예외 던지기 에러 처리


'OK'

In [99]:
next(g)

2

In [100]:
next(g)

first 제너레이터 None 값 수신


3

In [101]:
next(g)

first 제너레이터 None 값 수신


4

In [102]:
next(g)

first 제너레이터 None 값 수신
first 제너레이터 5에서 시작


5

In [103]:
g.throw(CustomException("두 번째 제너레이터를 향한 예외 던지기"))

first 제너레이터 두 번째 제너레이터를 향한 예외 던지기 에러 처리


'OK'

이 예제는 우리에게 많은 것을 시사한다. sequence 서브 제너레이터에 값을 보내지 않고 오직 main 제너레이터에 값을 보냈다는 것에 주목하자. 실제 값을 받는 것은 내부 제너레이터이다. 명시적으로 sequence에 데이터를 보낸 적은 없지만 실질적으로 yield from을 통해 sequence에 데이터를 전달한 셈이다. 

main 코루틴은 내부적으로 두 개의 다른 코루틴을 호출하여 값을 생산하며 특정 시점에서 보면 둘 중 하나는 멈춰져 있는 상태다. 로그를 통해 첫 번째 코루틴이 멈춰진 상태에서 데이터를 전송해도 첫 번째 코루틴 인스턴스가 값을 받는다는 것을 알 수 있다. 예외를 던질 때도 마찬가지다. 첫 번째 코루틴이 끝나면 step1 변수에 값을 반환하고, 그 값을 두 번째 코루틴에 입력으로 전달한다. 두 번째 코루틴도 첫 번째 코루틴과 동일하게 send()와 throw()에 대해 동일한 작업을 한다.

각 코루틴이 생성하는 값에 대해서도 마찬가지이다. 특정 단계에서 send()를 호출했을 떄 생성하는 값은 사실 현재 main 제너레이터가 멈춰 있던 서브 코루틴에서 생선한 값이다. 처리 가능한 CustomException 예외를 던지면 sequence 코루틴에서 OK를 생산하며 호출자 코루틴인 main에 전파한다.

### 비동기 프로그래밍

지금까지 살펴본 것들을 활용해 파이썬에서 비동기 프로그램을 만들 수 있다. 즉 여러 코루틴이 특정 순서로 동작하도록 스케줄링을 할 수 있으며, 일시 정지된 yield from 코루틴과 통신할 수 있다.

이러한 기능을 통해 얻을 수 있는 가장 큰 장점은 논블로킹(non-blocking) 방식으로 병렬 I/O 작업을 할 수 있다는 것이다. 이 떄 필요한 것은 보통 서드파티 라이브러리에서 구현한 저수준의 제너레이터이다. 이 라이브러리들은 코루틴이 일시 중단된 동안 실제 I/O 처리를 한다. 코루틴이 정지된 동안 프로그램은 다른 작업을 할 수 있어 효율적이다. 프로그램은 yield from 문장에 의해 중단되기도 하고 생산된 값을 받기도 하며 제어권을 주고 받는다.

비동기 프로그래밍을 지원하기 위한 더 나은 구문을 지원하기 전까지 실제로 파이썬에서는 이런 식으로 비동기 기능을 구현했었다,

코루틴과 제너레이터가 기술적으로 동일하다는 점에서 혼란스러울 때가 있다. 문법적으로(또는 기술적으로) 이들은 동일하지만 의미적으로는 다르다. 효율적인 반복을 원할 떄는 제너레이터를 사용하고, 논블로킹 I/O 작업을 원할 떄는 코루틴을사용한다.

차이점은 분명하지만 파이썬의 동적 특성으로 인해 이러한 객체를 혼합해서 사용하다가 개발 마지막 단계에서 런타임 오류가 발생하기도 한다. 앞서 yield from 구문을 사용한 간단한 예제로 제시했던 chain 함수를 기억해보자. 이 객체들은 사실 코루틴이 아니었지만 문제가 없이 잘 동작했다. 그 다음 yield from을 사용해 여러 코루틴에게 값을 보내고 예외를 던지고 값을 가져오는 것도 살펴보았다. 이 둘은 명백히 다른 성격의 사용 예제였다. 그런데 다음과 같은 코드가 있다고 가정해보자.

result = yield from iterable_or_awaitable()

iterable_or_awaitable이 반환하는 것이 명확하지 않다. 단순히 문자열과 같은 이터러블이어도 문법상 문제가 없다. 또는 실제 코루틴일 수도 있다. 이러한 유형의 실수는 나중에 큰 비용을 초래하기 마련이다. 

이런 이유 때문에 파이썬의 타이핑 시스템이 확장되었다. 파이썬 3.5 이전에 코루틴은 @coroutine 데코레이터가 적용된 제너레이터일 뿐이었으며 yield from 구문을 사용해 호출했었다. 그러나 이제 코루틴이라는 새로운 타입이 추가되었다.

새로운 구문으로 await와 async def 또한 추가되었다. await는 yield from을 대신하기 위한 용도로 awaitable 객체에 대해서만 동작한다. 코루틴은 awaitable이다. awaitable 인터페이스를 따르지 않는 객체에 await를 호출하면 예외가 발생한다. async def는 @coroutine 데코레이터를 대신하여 코루틴을 정의하는 새로운 방법이다. 이것은 호출 시 실제로 객체를 만들어 코루틴 인스턴스를 반환한다.

파이썬에서 비동기 프로그래밍을 한다는 것은 일련의 코루틴을 관리하는 이벤트 루프가 있다는 뜻이다(이벤트 루프라 하면 지금은 표준 라이브러리에 추가된 asyncio를 뜻하지만 동일한 일을 할 수 있는 다른 라이브러리가 많이 있다). 일련의 코루틴들은 이벤트 루프에 속하며, 이벤트 루프의 스케줄링 매커니즘에 따라 호출된다. 각각의 코루틴이 실행되면 사용자가 작성한 내부 코드가 실행되고 다시 이벤트 루프에 제어권을 반납하면 await coroutine을 호출하면 된다. await coroutine는 yield처럼 호출자에 제어권을 넘겨줌으로써 이벤트 루프가 작업을 비동기적으로 관리할 수 있게 해준다.

async / await에서 말하는 병렬이라는 것은 DB나 네트워크, 디스크 요청과 같은 Non-CPU 중심의 작업에 대해서 대기 시간이 생기면 제어권을 스케줄러에 넘겨줌으로써 그 사이에 다른 작업을 할 수 있게 한다는 뜻이다. 이런 식으로 각각의 작업(코루틴)이 실제로 CPU를 사용하는 시간은 매우 짧고 I/O 작업의 완료를 이벤트 형태로 알려줄 수 있다면 단일 스레드에서 마치 여러 개를 동시에 실행한 것과 비슷한 효과를 낼 수 있다는 뜻이다.